In [0]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        print("init resnet")

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        self.linearHPP = nn.Linear(256, num_classes)
        self.downsample=nn.Conv2d(2048, 256, kernel_size=1, stride=1, bias=False)
        self.maxpool1=nn.MaxPool2d((4,4), stride=1)
        self.maxpool2=nn.MaxPool2d((2,4), stride=1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
      
    def HPP(self,out):
#         Hyper Pyramid Pooling Layer.
#         Pooling to get G
        pool11= F.avg_pool2d(out,4)+self.maxpool1(out)
        pool21= F.avg_pool2d(out[:,:,0:2,:],(2,4))+self.maxpool2(out[:,:,0:2,:])
        pool22= F.avg_pool2d(out[:,:,2:4,:],(2,4))+self.maxpool2(out[:,:,2:4,:])
#         Downsampling the dimensions from 2048 to 256 to get H
      
        H22=self.downsample(pool22)
        H11=self.downsample(pool11)
        H21= self.downsample(pool21)
      
#     Flattening out the H for final classication
        flattened22=H22.view(H22.size(0),-1)
        flattened21=H21.view(H21.size(0),-1)
        flattened11=H11.view(H11.size(0),-1)
      
#       Concatenating the results for final output
        finalOutput=torch.cat((self.linearHPP(flattened11),self.linearHPP(flattened21),self.linearHPP(flattened22)),0)
        
        return finalOutput
      

    def forward(self, x):

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return self.HPP(out)





def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])





In [8]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse



device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

batchsize=32
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, shuffle=True, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net = ResNet50()

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    it=0
    max_accuracy=0
    final_loss=0
    
    net.train()

    for batch_idx, (inputs, targets) in enumerate(trainloader):

        it=it+1
        
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        batch=(targets.size())[0]
        if(batch<32):
          print(batch)
          print(outputs.size())
        HPPTargets=torch.cat((targets,targets,targets),0)

        loss=criterion(outputs[0:batch],targets)

        for i in range(0,2):
          loss= loss+ criterion(outputs[batch*(i+1):batch+(batch*(i+1))],targets)


        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += HPPTargets.size(0)
        correct += predicted.eq(HPPTargets).sum().item()
        
        batch_total=HPPTargets.size(0)
        batch_correct= predicted.eq(HPPTargets).sum().item()
        batch_accuracy= 100.*batch_correct/batch_total
        final_loss=train_loss/(batch_idx+1)
        
        if batch_accuracy>max_accuracy:
          max_accuracy=batch_accuracy

    print('Loss: %.3f | Max Batch Accuracy: %.3f | Epoch Accuracy : (%.3f)'% (final_loss, max_accuracy , 100.*correct/total))
        
        



for epoch in range(0,40):
  
  train(epoch)
  


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..
init resnet

Epoch: 0
16
torch.Size([48, 10])
Loss: 5.345 | Max Batch Accuracy: 71.875 | Epoch Accuracy : (33.685)

Epoch: 1
16
torch.Size([48, 10])
Loss: 3.831 | Max Batch Accuracy: 82.292 | Epoch Accuracy : (54.220)

Epoch: 2
16
torch.Size([48, 10])
Loss: 3.000 | Max Batch Accuracy: 91.667 | Epoch Accuracy : (65.224)

Epoch: 3
16
torch.Size([48, 10])
Loss: 2.435 | Max Batch Accuracy: 93.750 | Epoch Accuracy : (72.051)

Epoch: 4
16
torch.Size([48, 10])
Loss: 2.078 | Max Batch Accuracy: 96.875 | Epoch Accuracy : (76.365)

Epoch: 5
16
torch.Size([48, 10])
Loss: 1.823 | Max Batch Accuracy: 96.875 | Epoch Accuracy : (79.515)

Epoch: 6
16
torch.Size([48, 10])
Loss: 1.635 | Max Batch Accuracy: 100.000 | Epoch Accuracy : (81.506)

Epoch: 7
16
torch.Size([48, 10])
Loss: 1.484 | Max Batch Accuracy: 100.000 | Epoch Accuracy : (83.265)

Epoch: 8
16
torch.Size([48, 10])
Loss: 1.37